# A step-by-step tutorial of pydocker usage
## Main difference between gpu and non-gpu is a extra parameter 'devices' in docker.run()

In [15]:
from dsd.sys.docker.pydocker import pydocker
from io import BytesIO
import os

In [16]:
docker = pydocker()

init pydocker


In [17]:
docker

<dsd.sys.docker.pydocker.pydocker instance at 0x7f47824d7050>

# 1. Build
## default dockerfile name is $Dockerfile$
Dockerfile:
```
# For Test
FROM ubuntu:14.04
MAINTAINER zy <zy3381@gmail.com>
EXPOSE 9999
```
## returns: build logs

In [18]:
imageTag = 'joe/buildgputest:cuda7.0'
response = docker.build(tag=imageTag, dockerfilePath='../docker/test_gpu')
if response:
    for line in response:
        print line
else:
    print 'build failed'

# For Test
FROM guoq/imxnet:cuda-7.0
MAINTAINER zy <zy3381@gmail.com>
EXPOSE 9999

{"stream":"Step 1 : FROM guoq/imxnet:cuda-7.0\n"}

{"stream":" ---\u003e df99d4c54180\n"}

{"stream":"Step 2 : MAINTAINER zy \u003czy3381@gmail.com\u003e\n"}

{"stream":" ---\u003e Using cache\n"}

{"stream":" ---\u003e 1df6aafa99e5\n"}

{"stream":"Step 3 : EXPOSE 9999\n"}

{"stream":" ---\u003e Using cache\n"}

{"stream":" ---\u003e a3799a1ff216\n"}

{"stream":"Successfully built a3799a1ff216\n"}



# 2. list images

In [19]:
pses = docker.images()
for line in pses:
    print line['RepoTags']

joe/buildgputest:cuda7.0
joe/test1:latest
wzz/lasagne:latest
kaixhin/cuda-theano:7.0
shawn/cuda-mxnet:422
<none>:<none>
<none>:<none>
cuda-mxnet:20160421
kaixhin/cudnn:7.0
wangjianyong/mxnet_jupyter:latest
caffe:latest
guoq/imxnet:cuda-7.0
<none>:<none>
ubuntu:14.04
hello-world:latest
kaixhin/cuda-digits:latest
kaixhin/cuda:7.0


# 3. run
## $imageTag$ comes from step 1
## params:
## name='myUbuntuTest',
==============================================================
## ports={8888:1234, 7777:5678},   type:dict
#### net stat: *9999/tcp, 0.0.0.0:5678->7777/tcp, 0.0.0.0:1234->8888/tcp*
==============================================================
## volumes={'/home/zhouyao/':'/mnt/homeOfJoe'},   type:dict
#### volume in host: */home/zhouyao/*, volume in container: */mnt/homeOfJoe*
==============================================================
## GPU devices: specified four gpu cards(nvidia0-3),  type:list

#### tests below all successed in mxnet 

```
import mxnet as mx
a1 = mx.ndarray.ones([1,1], ctx=mx.gpu(0))
a2 = mx.ndarray.ones([1,1], ctx=mx.gpu(1))
a3 = mx.ndarray.ones([1,1], ctx=mx.gpu(2))
a4 = mx.ndarray.ones([1,1], ctx=mx.gpu(3))
```

In [20]:
container = docker.run(image=imageTag, 
           detach=True, 
           stdin_open=True, 
           tty=True, 
           command='bash', 
           name='myGpuTest',
           user=None,
                       
           ports={8888:1234, 7777:5678},
                       
           volumes={'/home/zhouyao/':'/mnt/homeOfJoe'},
                       
           devices=['/dev/nvidia-uvm', '/dev/nvidiactl', '/dev/nvidia0', '/dev/nvidia1', '/dev/nvidia2', '/dev/nvidia3']
           )
if container is not None:
    print 'container is runing with id:'
    print container.get('Id')
else:
    print 'create container error'

container is runing with id:
1d2ef803dce8774db857e9c86081e3690edf8229cec85e331604ffb4b2c7fd39


# 4. start
## *container_id* comes from step 3

In [21]:
container_id = container.get('Id')
response = docker.start(container=container_id)
if response:
    print 'container ', container_id, ' started'
else:
    print 'starting container error'

container  1d2ef803dce8774db857e9c86081e3690edf8229cec85e331604ffb4b2c7fd39  started


# 5. check status

In [22]:
pses = docker.ps()
print 'Runing containers:'
print '##################'
for line in pses:
    print line['image'], line['name']

Runing containers:
##################
joe/buildgputest:cuda7.0 /myGpuTest
wzz/lasagne /wzz_docker
guoq/imxnet:cuda-7.0 /lwb-mxnet
sha256:c832a806ee999c1091fc279fd427bf331e838c57789e21d9d59ed4b98826c377 /wjy-mxnet


# 6. Stop 

In [23]:
container_id = container.get('Id')
response = docker.stop(container=container_id,
                       timeout=0)
if response:
    print 'container', container_id, 'stop succeed'
else:
    print '#######################'
    print 'container stop failed'

container 1d2ef803dce8774db857e9c86081e3690edf8229cec85e331604ffb4b2c7fd39 stop succeed


# 7. Remove container

In [24]:
container_id = container.get('Id')
response = docker.rm(container=container_id)
if response:
    print 'container ', container_id, 'removed succed'
else:
    print '#######################'
    print 'container removed failed'

container  1d2ef803dce8774db857e9c86081e3690edf8229cec85e331604ffb4b2c7fd39 removed succed


# 8. Remove image

In [25]:
response = docker.rmi(image=imageTag,
                      force=False,
                      noprune=None)
if response:
    print 'image ', imageTag, 'removed succeed'
else:
    print '#######################'
    print 'image removed failed'

image  joe/buildgputest:cuda7.0 removed succeed
